In [4]:
# Dataset
from sklearn.datasets import load_diabetes

# Basics
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Model
from sklearn.ensemble import RandomForestRegressor

# Helpful:
from sklearn.model_selection import train_test_split

import sys
sys.path.append("/home/dchen/Random_Forest_Weights/")
# Now, you can import your modules
from src_rf.methods.calc_mean import *
from src_rf.methods.calc_weights import *
from src_rf.methods.calc_dist import *

### 1. Loading Data & Train_test_split

In [16]:
diabetes = load_diabetes()
df_diabetes = pd.DataFrame(
    data=np.c_[diabetes["data"], diabetes["target"]],
    columns=diabetes["feature_names"] + ["target"],
)

In [17]:
X = df_diabetes.iloc[:, :-1].to_numpy(dtype="float32")
y = df_diabetes.iloc[:, -1:].to_numpy().flatten()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### 2. Random Forest

In [20]:
bootstrap = True
max_sample = 0.7

In [21]:
rf = RandomForestRegressor(bootstrap=bootstrap, max_samples=max_sample)

In [22]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_samples=0.7)

### 3. Calculate Weights Random Forest

In [23]:
rf_weights = calc_weights_rf(rf, X_train, X_test, bootstrap, max_sample)

100%|██████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 260.63it/s]


In [26]:
rf_weights.shape

(100, 89, 353)

In [27]:
X_test.shape

(89, 10)

In [29]:
X_train.shape

(353, 10)

### 4. Calc Mean with weights and normal:

In [30]:
rf_mean_weights = calc_mean_rf(rf_weights, y_train)
rf_mean_normal = rf.predict(X_test)

In [32]:
# Are the two the same?
sum(np.round(rf_mean_weights, 10) == np.round(rf_mean_normal, 10))

89

### 5. Calc Quantile Random Forest

In [33]:
rf_cdfs = calc_dist_rf(rf_weights, y_train)

In [35]:
len(rf_cdfs)

89

In [36]:
rf_median = calc_quantile_rf(rf_cdfs, 0.5, y_train)

In [37]:
rf_median[:10]

[236.0000000000008,
 244.125,
 92.99999999999977,
 219.90625,
 92.99999999999977,
 280.9999999999995,
 85.4921875,
 76.99999999999892,
 65.00000000000102,
 239.28125]